<a href="https://colab.research.google.com/github/ulisses-adonis/.Artificial-neural-network--projects/blob/main/Alexnet_CYFAR100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras.datasets import cifar10, cifar100
from keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import numpy as np

In [2]:
np.random.seed(1000)
(X_train10, Y_train10), (X_test10, Y_test10) = cifar10.load_data()
(X_train100, Y_train100), (X_test100, Y_test100) = cifar100.load_data()

Um dos problemas de aplicar o AlexNet diretamente no dataset CIFAR-10 ou CIFAR100 é que as  imagens desses dois conjunto de imagens,  têm resolução inferior (32 × 32 pixels) em relação as imagens usadas no dataset original (Image Net), para qual a AlexNet foi proposta . Para fazer as coisas funcionarem, deveríamos aumentar a resolução da amostra para 224 × 224 pixels (geralmente não é uma prática inteligente, mas faríamos isso para sermos fiéis à arquitetura AlexNet). 

In [3]:
alexnet = Sequential()

# Layer 1
alexnet.add(Conv2D(96, (3, 3), input_shape=(32, 32, 3),
 padding='same'))
alexnet.add(BatchNormalization())
alexnet.add(Activation('relu'))
alexnet.add(MaxPooling2D(pool_size=(2, 2)))
# Layer 2
alexnet.add(Conv2D(256, (5, 5), padding='same'))
alexnet.add(BatchNormalization())
alexnet.add(Activation('relu'))
alexnet.add(MaxPooling2D(pool_size=(2, 2)))
# Layer 3
alexnet.add(Conv2D(512, (3, 3), padding='same'))
alexnet.add(BatchNormalization())
alexnet.add(Activation('relu'))
alexnet.add(MaxPooling2D(pool_size=(2, 2)))
# Layer 6
alexnet.add(Flatten())
alexnet.add(Dense(4096))
alexnet.add(BatchNormalization())
alexnet.add(Activation('relu'))
alexnet.add(Dropout(0.5))
# Layer 8
alexnet.add(Dense(100))
alexnet.add(BatchNormalization())
alexnet.add(Activation('softmax'))

O tamanho do lote (batch_size) é um dos hiperparâmetros mais importantes para sintonizar os modernos sistemas de aprendizagem profunda. Os Cientistas de Dados muitas vezes querem usar um tamanho de lote maior para treinar seu modelo, uma vez que permite acelerações computacionais do paralelismo das GPUs. No entanto, é bem conhecido que um tamanho de lote muito grande levará a uma generalização deficiente (embora atualmente não se saiba exatamente porque isso acontece). 	No caso  com um bacth_size =64, para  5 épocas , o modelo obteve uma precisão melhor do que para um batch_size = 128 e 10 épocas.

In [4]:
alexnet.compile(loss='categorical_crossentropy',
 optimizer=Adam(lr=0.0001),
 metrics=['accuracy'])



/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [5]:
alexnet.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 96)        2688      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 96)        384       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 96)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 256)       614656    
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 256)       1024      
_________________________________________________________________
activation_1 (Activation)    (None, 16, 16, 256)       0

In [6]:
#(X_train100, Y_train100), (X_test100, Y_test100) = cifar100.load_data()
#alexnet.fit(X_train10 / 255.0, to_categorical(Y_train10),
alexnet.fit(X_train100 / 255.0, to_categorical(Y_train100),
 batch_size=64,
 shuffle=True,
 epochs=10,
#validation_data=(X_test10 / 255.0, to_categorical(Y_test10)))
validation_data=(X_test100/ 255.0, to_categorical(Y_test100)))

Epoch 1/10
782/782 [==============================] - 40s 29ms/step - loss: 3.7235 - accuracy: 0.1661 - val_loss: 2.9423 - val_accuracy: 0.3313
Epoch 2/10
782/782 [==============================] - 22s 28ms/step - loss: 2.7455 - accuracy: 0.3767 - val_loss: 2.6108 - val_accuracy: 0.4123
Epoch 3/10
782/782 [==============================] - 22s 28ms/step - loss: 2.3505 - accuracy: 0.4852 - val_loss: 2.3884 - val_accuracy: 0.4510
Epoch 4/10
782/782 [==============================] - 22s 28ms/step - loss: 2.0381 - accuracy: 0.5707 - val_loss: 2.2968 - val_accuracy: 0.4851
Epoch 5/10
782/782 [==============================] - 22s 28ms/step - loss: 1.7265 - accuracy: 0.6726 - val_loss: 2.3567 - val_accuracy: 0.4788
Epoch 6/10
782/782 [==============================] - 22s 28ms/step - loss: 1.4572 - accuracy: 0.7587 - val_loss: 2.2892 - val_accuracy: 0.4954
Epoch 7/10
782/782 [==============================] - 22s 28ms/step - loss: 1.1936 - accuracy: 0.8412 - val_loss: 2.2022 - val_accuracy:

Os valores de accuracy e val_accuracy diferem porque o primeiro é aplicado ao conjunto de treino e o último ao conjunto de teste.  Em resumo, accuracy é a precisão de um lote de dados de treinamento e val_acc é a precisão de um lote de dados de teste.
Como tal, o último é uma boa indicação do desempenhodo modelo em dados não vistos.  É melhor contar com val_acc para uma representação justa do desempenho do modelo porque uma boa rede neural acabará ajustando os dados de treinamento a 100%, mas teria um desempenho insatisfatório em dados invisíveis

In [7]:
scores = alexnet.evaluate(X_test100 / 255.0, to_categorical(Y_test100))


313/313 [==============================] - 2s 5ms/step - loss: 2.2638 - accuracy: 0.5039


In [8]:
print('Loss: %.3f' % scores[0])
print('Accuracy: %.3f' % scores[1])

Loss: 2.264
Accuracy: 0.504


 **Rede com  batch_size = 128**   

 Loss: 2.645 Accuracy: 0.402

